In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import altair as alt
import polars as pl


In [3]:
from plan2eplus.studies.analysis2.all_cases_v_time import get_temp_dfs
from plan2eplus.studies.experiments.retrieve import retrieve_control_cases

In [5]:
cases = retrieve_control_cases()
df, df_site = get_temp_dfs(cases)

In [21]:
print(df.head())
# print(df_site.head())


shape: (5, 5)
┌──────────────┬───────────────────┬─────────────────────┬───────────────────────────────┬───────┐
│ case_names   ┆ space_names       ┆ datetimes           ┆ Zone Mean Air Temperature [C] ┆ time  │
│ ---          ┆ ---               ┆ ---                 ┆ ---                           ┆ ---   │
│ str          ┆ str               ┆ datetime[μs]        ┆ f64                           ┆ str   │
╞══════════════╪═══════════════════╪═════════════════════╪═══════════════════════════════╪═══════╡
│ bol_5_Medium ┆ BLOCK 00 STOREY 0 ┆ 2017-07-01 00:00:00 ┆ 19.089994                     ┆ 00:00 │
│ bol_5_Medium ┆ BLOCK 00 STOREY 0 ┆ 2017-07-01 00:15:00 ┆ 18.163066                     ┆ 00:15 │
│ bol_5_Medium ┆ BLOCK 00 STOREY 0 ┆ 2017-07-01 00:30:00 ┆ 17.244493                     ┆ 00:30 │
│ bol_5_Medium ┆ BLOCK 00 STOREY 0 ┆ 2017-07-01 00:45:00 ┆ 16.338522                     ┆ 00:45 │
│ bol_5_Medium ┆ BLOCK 00 STOREY 0 ┆ 2017-07-01 01:00:00 ┆ 16.150287                     ┆ 01:0

In [29]:
df["space_names"].unique()

space_names
str
"""BLOCK 09 STOREY 0"""
"""BLOCK 010 STOREY 0"""
"""BLOCK 06 STOREY 0"""
"""BLOCK 01 STOREY 0"""
"""BLOCK 07 STOREY 0"""
…
"""BLOCK 00 STOREY 0"""
"""BLOCK 03 STOREY 0"""
"""BLOCK 04 STOREY 0"""


In [28]:
df["datetimes"].unique()

datetimes
datetime[μs]
2017-07-01 00:00:00
2017-07-01 00:15:00
2017-07-01 00:30:00
2017-07-01 00:45:00
2017-07-01 01:00:00
…
2017-07-01 22:45:00
2017-07-01 23:00:00
2017-07-01 23:15:00


In [35]:
df_agg = df.group_by(["space_names", "datetimes", "case_names"]).agg( pl.col("Zone Mean Air Temperature [C]").mean())
df_agg

space_names,datetimes,case_names,Zone Mean Air Temperature [C]
str,datetime[μs],str,f64
"""BLOCK 05 STOREY 0""",2017-07-01 16:30:00,"""red_b1_Medium""",27.437929
"""BLOCK 08 STOREY 0""",2017-07-01 21:45:00,"""amb_b1_Medium""",22.617261
"""BLOCK 06 STOREY 0""",2017-07-01 06:45:00,"""red_b1_Medium""",19.645753
"""BLOCK 09 STOREY 0""",2017-07-01 15:15:00,"""red_b1_Medium""",27.941313
"""BLOCK 01 STOREY 0""",2017-07-01 17:15:00,"""amb_b1_Medium""",26.86257
…,…,…,…
"""BLOCK 06 STOREY 0""",2017-07-01 03:15:00,"""amb_b1_Medium""",18.287429
"""BLOCK 01 STOREY 0""",2017-07-01 14:00:00,"""red_b1_Medium""",28.40166
"""BLOCK 01 STOREY 0""",2017-07-01 18:45:00,"""red_b1_Medium""",23.858379


In [36]:
alt.Chart(df_agg).mark_line().encode(
    x='datetimes:T',
    y=alt.Y(r'Zone Mean Air Temperature \[C\]:Q').scale(zero=False),
    color='case_names:N',
).transform_aggregate(
    last_date="argmax(date)",
    groupby=["symbol"]
)

alt.Chart(...)